In [3]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve, curve_fit
from scipy.integrate import odeint, quad
from scipy.optimize import minimize_scalar
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

In [4]:
class honolulu(object):
    def __init__(self, a, b, c, n):
        self.n = n
        self.a = a
        self.b = b
        self.c = c

    # F(x) = x
    # G(x) = F(x) ^ (n - 1)
    # g(x) = G'(x)
    # H(x) = integral(y[0, x])(G(y))
    # G(v, x) = n * F(v) * F(x) ^ (n - 1)
    # g(v, x) = d^2 G(v, x) / dvdx

    def K(self, pvar, v, s):
        # K(p, v, s) = 2 * b * (H(v) - H(p)) - G(p) * (a - b * (s - v))
        return (
            2 * self.b 
            * (v ** self.n - pvar ** self.n) / self.n 
            - pvar ** (self.n - 1) 
            * (self.a - self.b * (s - v))
        )

    def k(self, v, s):
        # p = k(v, s) solves K(p, v, s) = 0
        K_fun = lambda pvar: self.K(pvar, v, s)
        sol = fsolve(K_fun, v / 2)[0]
        return sol

    def p(self, v, s):
        # p = min{s, k(v, s)}
        return np.minimum(s, self.k(v, s))

In [5]:
df_h = pd.read_csv("Honolulu_paydiff_norm.csv")

## Cost Estimation

In [6]:
def estCost(n, b, s, tol):

    tmp = df_h[df_h["session.config.name"].str.contains(str(n) + "_") & (df_h["session.config.discount_b"] == b)]
    tmp = tmp[
        (tmp["player.is_dutch_winner"] == 1)
        # & (
        #     ((tmp["group.have_contest_winner"] > 1) & (tmp["player.is_english_winner"] == 0) & (abs(tmp["player.dropout_price"] - tmp["player.item_value"]) <= tol))
        #     | ((tmp["group.have_contest_winner"] > 1) & (tmp["player.is_english_winner"] == 1) & (tmp["group.final_price"] <= tmp["player.item_value"] + tol))
        #     | ((tmp["group.have_contest_winner"] == 1) & (tmp["group.final_price"] <= tmp["player.item_value"] + tol))
        # )
    ]
    ha = lambda x: honolulu(a=1, b=x, c=0.95, n=n)
    
    # categorised for all sessions
    tmp_dutchhigh = tmp[tmp["group.dutch_final_price"] > tmp["predict.player.optimal_dutch_bid"] + tol]
    tmp_dutchlow = tmp[tmp["group.dutch_final_price"] < tmp["predict.player.optimal_dutch_bid"] - tol]
    tmp_dutchmid = tmp[abs(tmp["group.dutch_final_price"] - tmp["predict.player.optimal_dutch_bid"]) <= tol]
    for tar_df, name_df in [(tmp_dutchhigh, "high"), (tmp_dutchmid, "mid"), (tmp_dutchlow, "low")]:
        xdata = (tar_df["player.item_value"] / 50).tolist()
        ydata = (tar_df["group.dutch_final_price"] / 50).tolist()
        fit = curve_fit(lambda x, param: list(map(lambda v: ha(param).p(v, s), x)), xdata, ydata)
        print([fit[0], np.sqrt(np.diag(fit[1])), name_df, len(tar_df)])

    # pooled for all sessions
    xdata = (tmp["player.item_value"] / 50).tolist()
    ydata = (tmp["group.dutch_final_price"] / 50).tolist()
    fit = curve_fit(lambda x, param: list(map(lambda v: ha(param).p(v, s), x)), xdata, ydata)
    print([fit[0], np.sqrt(np.diag(fit[1])), "all sessions", len(tmp)])
    
    # pooled for each session
    sessions = set(tmp["session.code"])
    for scode in sessions:
        tar_df = tmp[tmp["session.code"] == scode]
        xdata = (tar_df["player.item_value"] / 50).tolist()
        ydata = (tar_df["group.dutch_final_price"] / 50).tolist()
        fit = curve_fit(lambda x, param: list(map(lambda v: ha(param).p(v, s), x)), xdata, ydata)
        print([fit[0], np.sqrt(np.diag(fit[1])), scode, len(tar_df)])

In [7]:
estCost(n=2, b=0.019, s=0.42, tol=2)

c:\Program Files\Python39\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


[array([2.63156979]), array([0.25485942]), 'high', 82]
[array([0.91522251]), array([0.01906345]), 'mid', 90]
[array([0.26995155]), array([0.01556615]), 'low', 162]
[array([0.485114]), array([0.0268199]), 'all sessions', 334]
[array([0.3531888]), array([0.04136222]), 'nli0vb0t', 85]
[array([0.99962667]), array([0.14706164]), 'cbdt6t3s', 76]
[array([0.47142692]), array([0.04603003]), 'p66lw61j', 86]
[array([0.44005224]), array([0.04451825]), '7xbozwc7', 87]


In [8]:
estCost(n=2, b=0.009, s=0.24, tol=2)

[array([1.63912526]), array([0.13807635]), 'high', 72]
[array([0.43796521]), array([0.01255024]), 'mid', 105]
[array([0.13032179]), array([0.00735719]), 'low', 136]
[array([0.20308989]), array([0.01180805]), 'all sessions', 313]
[array([0.12752372]), array([0.01816312]), 'c9oliyzc', 66]
[array([0.20376082]), array([0.02384294]), 'yq06059v', 86]
[array([0.23104475]), array([0.02768335]), '5m969a1u', 68]
[array([0.24869403]), array([0.02274156]), '785y9gdo', 93]


In [9]:
estCost(n=5, b=0.019, s=0.64, tol=2)

[array([3.39665927]), array([0.42519928]), 'high', 85]
[array([0.99223035]), array([0.04746669]), 'mid', 59]
[array([0.2113252]), array([0.02794577]), 'low', 66]
[array([1.10738455]), array([0.12049523]), 'all sessions', 210]
[array([2.13205125]), array([0.39249097]), 'w4lqxa01', 56]
[array([0.32051991]), array([0.07886986]), '15t0rnnr', 50]
[array([2.9087749]), array([7.98514693e-06]), 'ogo32l47', 54]
[array([1.0551554]), array([0.19603922]), 'btnhfeao', 50]


In [11]:
estCost(n=5, b=0.009, s=0.54, tol=2)

[array([2.46813557]), array([0.45252292]), 'high', 57]
[array([0.90395783]), array([0.05119306]), 'mid', 88]
[array([0.2109055]), array([0.01692765]), 'low', 51]
[array([2.12201104]), array([0.25931037]), 'all sessions', 196]
[array([2.12201099]), array([0.40665044]), 'r6ovli5x', 84]
[array([0.81163989]), array([0.14289323]), 'a2ehf0h4', 56]
[array([2.89207132]), array([0.4830369]), 'hie4719f', 56]


## Bidder Types

In [52]:
tol = 2

# dutch bid too low (delay), either the dutch winner bids too late or the dutch non-winner miss the chance to bid optimally 
df_h["dutch_low"] = (df_h["group.dutch_final_price"] < df_h["predict.player.optimal_dutch_bid"] - tol)

# dutch winner bid too high
df_h["dutch_high"] = (
    (df_h["player.is_dutch_winner"] == 1) 
    & (df_h["group.dutch_final_price"] > df_h["predict.player.optimal_dutch_bid"] + tol)
)

# competitive dutch decision
df_h["dutch_mid"] = (
    (df_h["dutch_low"] == 0) 
    & (df_h["dutch_high"] == 0)
)

# buy the item lower than group-level prediction 
df_h["buy_low"] = (
    (df_h["player.is_final_winner"] == 1)
    & (df_h["player.dropout_price"] < df_h["predict.group.final_price"] - tol)
)

# drop out below the item value
df_h["drop_low"] = (
    (df_h["player.is_final_winner"] == 0)
    & (df_h["player.dropout_price"] < df_h["player.item_value"] - tol)
)

# join the contest but drop out above the item value, either wrong contest join decision or staying too long in the english stage
df_h["overbid"] = (
    (df_h["player.contest_status"] == 1) 
    & (df_h["player.dropout_price"] > df_h["player.item_value"] + tol)
)

# competitive drop or buy
df_h["competitive"] = (
    (df_h["buy_low"] == 0)
    & (df_h["drop_low"] == 0)
    & (df_h["overbid"] == 0)
)

In [56]:
def estBidderCost(key1, key2):
    print("%s, %s" % (key1, key2))
    print("est_b\tstd\tobs\ttreatment")
    for n, b, s in [(2, 0.019, 0.42), (2, 0.009, 0.24), (5, 0.019, 0.64), (5, 0.009, 0.54)]:
        tmp = df_h[
            df_h["session.config.name"].str.contains(str(n) + "_") 
            & (df_h["session.config.discount_b"] == b)
            & (df_h["player.is_dutch_winner"] == 1)
            & (df_h[key1] == 1)
            & (df_h[key2] == 1)
        ]
        ha = lambda x: honolulu(a=1, b=x, c=0.95, n=n)
        if len(tmp) > 0:
            xdata = (tmp["player.item_value"] / 50).tolist()
            ydata = (tmp["group.dutch_final_price"] / 50).tolist()
            fit = curve_fit(lambda x, param: list(map(lambda v: ha(param).p(v, s), x)), xdata, ydata)
            print("%.2f\t%.2f\t%d\t%s" % (fit[0], np.sqrt(np.diag(fit[1])), len(tmp), str(n) + ("H" if b == 0.019 else "L")))

In [58]:
estBidderCost("dutch_low", "buy_low")
estBidderCost("dutch_low", "drop_low")
estBidderCost("dutch_low", "overbid")
estBidderCost("dutch_low", "competitive")

dutch_low, buy_low
est_b	std	obs	treatment
0.27	0.02	78	2H
0.12	0.01	49	2L
0.22	0.07	9	5H
0.22	0.03	10	5L
dutch_low, drop_low
est_b	std	obs	treatment
0.23	0.04	18	2H
0.09	0.02	11	2L
0.15	0.06	10	5H
0.25	0.09	4	5L
dutch_low, overbid
est_b	std	obs	treatment
0.46	0.10	2	2H
dutch_low, competitive
est_b	std	obs	treatment
0.28	0.02	64	2H
0.15	0.01	76	2L
0.22	0.04	47	5H
0.21	0.02	37	5L


In [60]:
estBidderCost("dutch_mid", "buy_low")
estBidderCost("dutch_mid", "drop_low")
estBidderCost("dutch_mid", "overbid")
estBidderCost("dutch_mid", "competitive")

dutch_mid, buy_low
est_b	std	obs	treatment
1.05	0.28	6	2H
0.37	0.03	8	2L
1.11	inf	1	5H
0.90	0.21	7	5L
dutch_mid, drop_low
est_b	std	obs	treatment
0.84	0.14	5	2H
0.39	0.03	10	2L
0.71	0.07	4	5H
0.79	0.17	7	5L
dutch_mid, overbid
est_b	std	obs	treatment
1.64	0.26	2	2H
0.41	inf	1	2L
0.96	inf	1	5H
1.17	0.56	2	5L
dutch_mid, competitive
est_b	std	obs	treatment
0.92	0.02	77	2H
0.45	0.01	86	2L
1.00	0.05	53	5H
0.95	0.06	72	5L


In [61]:
estBidderCost("dutch_high", "buy_low")
estBidderCost("dutch_high", "drop_low")
estBidderCost("dutch_high", "overbid")
estBidderCost("dutch_high", "competitive")

dutch_high, buy_low
est_b	std	obs	treatment
6.27	2.68	5	2H
1.07	0.14	7	2L
3.38	0.90	4	5H
2.58	inf	1	5L
dutch_high, drop_low
est_b	std	obs	treatment
1.91	0.44	3	2H
1.20	0.14	7	2L
2.37	inf	1	5H
2.00	0.07	4	5L
dutch_high, overbid
est_b	std	obs	treatment
5.56	5.08	10	2H
2.33	1.50	5	2L
3.40	2.12	13	5H
2.12	1.11	14	5L
dutch_high, competitive
est_b	std	obs	treatment
2.02	0.09	64	2H
1.80	0.18	53	2L
2.92	0.16	67	5H
3.90	0.28	38	5L
